# What make the XGBoost Extreme?

**Regularized loss function** :
1.  we generally use the regularization term to control the model complexity.If we only use the tree base algorithm as the weak leaner then we cna use the other technique to control the model complexity like we can specify the number of leaf node we need.
$$\lambda \cdot \overbrace{\mathbf{f}(\mathbf{x})^2}^{\text {measures model complexity }}+\gamma \cdot \overbrace{\mathbf{T}}^{\text {number of leaf nodes }}+\overbrace{\operatorname{loss}(\mathbf{f}(\mathbf{x}), \text { data })}^{\text {measures model fit }}$$
2. This regularized loss function is used to to find split in each nodes.
3. Adding the extra parameter comes with the extra parameter $\gamma$, for this we have to do the cross validation.

**Quantile based newton boosting**:
1. For scaling the large data use the weighted quantile is used.



# Sklearn API

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

X,y = load_breast_cancer(return_X_y=True)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,shuffle=True)

In [2]:
from xgboost import XGBClassifier
ens = XGBClassifier(n_estimators=120,max_depth=1,objective="binary:logistic")
ens.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=1, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=120,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [3]:
from sklearn.metrics import accuracy_score
test_error = 1 - accuracy_score(y_test,ens.predict(X_test))
test_error

0.03508771929824561

# Native Approach

XGBoost uses a data object called DMatrix to group data and corresponding labels together. DMatrix objects can be created by reading data directly from files or from other array-like objects. Here, we create two DMatrix objects called trn and tst to represent the train and test data matrices

In [4]:
import xgboost as xgb

train_data = xgb.DMatrix(data=X_train,label=y_train)
test_data = xgb.DMatrix(data=X_test,label=y_test)

In [5]:
# training params
params = dict(max_depth=1,objective="binary:logistic")
ens2 = xgb.train(params=params,
                 dtrain=train_data,
                 num_boost_round=20)

Care must be taken while using this model for prediction, however. Models trained with certain loss functions will return prediction probabilities rather than the predictions directly. The logistic loss function is one such case

In [7]:
y_pred_proba = ens2.predict(test_data)
y_pred =( y_pred_proba >= 0.5).astype(int)
1- accuracy_score(y_test,y_pred)

0.03508771929824561